In [ ]:
!rm -f watson_streaming_pipelines.py*
!wget https://raw.githubusercontent.com/raanonr/DSX/master/watson_streaming_pipelines.py

In [ ]:
# @hidden_cell
# The Object Storage credentials should NOT come from the 'Service credentials' page.
# Get them from the option in DSX Notebook to 'insert to code' from the Data -> Connections.
# Then add an entry for the service's PUBLIC endpoint.

cos_credentials_stage1_wd = {
  'iam_url':'https://iam.stage1.ng.bluemix.net/oidc/token',
  'api_key':'xhjheSC7AhSLtvapSDnbyFn17uWUqW5ccAOuHhQxnnEY',
  'resource_instance_id':'crn:v1:staging:public:cloud-object-storage:global:a/68a66698d275aeb48097f868957ab2ed:bbb5aa36-5525-4000-b129-bcb780195098::',
  'url':'https://s3-api.us-geo.objectstorage.uat.service.networklayer.com',
    'endpoint':'https://s3.us-west.objectstorage.uat.softlayer.net'
}
cos_credentials_prod_ki = {
  'iam_url':'https://iam.ng.bluemix.net/oidc/token',
  'api_key':'0s-JWmaDBwiSd_yWJqenoKRBfTVU5Rgkz31CDT5WgoWQ',
  'resource_instance_id':'crn:v1:bluemix:public:cloud-object-storage:global:a/db0d062d2b4c0836e18618a5222d8068:22e3b946-6154-4032-8e8f-7cfb0b429602::',
  'url':'https://s3-api.us-geo.objectstorage.service.networklayer.com',
      "endpoint":"https://s3-api.us-geo.objectstorage.softlayer.net",
}

# The Message Hub credentials should come from the 'Service credentials' page.

mh_credentials_stage1_1z = {
  "instance_id": "86ce5bfa-268e-4786-a7e9-91f8427f2e17",
  "mqlight_lookup_url": "https://mqlight-lookup-stage1.messagehub.services.us-south.bluemix.net/Lookup?serviceId=86ce5bfa-268e-4786-a7e9-91f8427f2e17",
  "api_key": "CBa3cYnfW1iWM25LNewJVZaxEP1qjKlThEeqz2Zrnkzu7VXd",
  "kafka_admin_url": "https://kafka-admin-stage1.messagehub.services.us-south.bluemix.net:443",
  "kafka_rest_url": "https://kafka-rest-stage1.messagehub.services.us-south.bluemix.net:443",
  "kafka_brokers_sasl": [
    "kafka02-stage1.messagehub.services.us-south.bluemix.net:9093",
    "kafka04-stage1.messagehub.services.us-south.bluemix.net:9093",
    "kafka01-stage1.messagehub.services.us-south.bluemix.net:9093",
    "kafka03-stage1.messagehub.services.us-south.bluemix.net:9093",
    "kafka05-stage1.messagehub.services.us-south.bluemix.net:9093"
  ],
  "user": "CBa3cYnfW1iWM25L",
  "password": "NewJVZaxEP1qjKlThEeqz2Zrnkzu7VXd"
}
mh_credentials_stage1_2s = {
  "instance_id": "81b7462e-7707-44c1-8bfa-8c9490ac8111",
  "mqlight_lookup_url": "https://mqlight-lookup-stage1.messagehub.services.us-south.bluemix.net/Lookup?serviceId=81b7462e-7707-44c1-8bfa-8c9490ac8111",
  "api_key": "phXq2H0NSDQNSCdKGJrEFTSnVHjgH8ugpChw1LgNbL3Sr23g",
  "kafka_admin_url": "https://kafka-admin-stage1.messagehub.services.us-south.bluemix.net:443",
  "kafka_rest_url": "https://kafka-rest-stage1.messagehub.services.us-south.bluemix.net:443",
  "kafka_brokers_sasl": [
    "kafka04-stage1.messagehub.services.us-south.bluemix.net:9093",
    "kafka05-stage1.messagehub.services.us-south.bluemix.net:9093",
    "kafka03-stage1.messagehub.services.us-south.bluemix.net:9093",
    "kafka01-stage1.messagehub.services.us-south.bluemix.net:9093",
    "kafka02-stage1.messagehub.services.us-south.bluemix.net:9093"
  ],
  "user": "phXq2H0NSDQNSCdK",
  "password": "GJrEFTSnVHjgH8ugpChw1LgNbL3Sr23g"
}
mh_credentials_prod_8x = {
  "instance_id": "3ba03d3f-158f-4016-9355-9447feb29a06",
  "mqlight_lookup_url": "https://mqlight-lookup-prod01.messagehub.services.us-south.bluemix.net/Lookup?serviceId=3ba03d3f-158f-4016-9355-9447feb29a06",
  "api_key": "Azrmak5KqB5aQ9YghzWI1bhxfxqB0ki7qXID3OlXOHPm54NX",
  "kafka_admin_url": "https://kafka-admin-prod01.messagehub.services.us-south.bluemix.net:443",
  "kafka_rest_url": "https://kafka-rest-prod01.messagehub.services.us-south.bluemix.net:443",
  "kafka_brokers_sasl": [
    "kafka01-prod01.messagehub.services.us-south.bluemix.net:9093",
    "kafka05-prod01.messagehub.services.us-south.bluemix.net:9093",
    "kafka03-prod01.messagehub.services.us-south.bluemix.net:9093",
    "kafka02-prod01.messagehub.services.us-south.bluemix.net:9093",
    "kafka04-prod01.messagehub.services.us-south.bluemix.net:9093"
  ],
  "user": "Azrmak5KqB5aQ9Yg",
  "password": "hzWI1bhxfxqB0ki7qXID3OlXOHPm54NX"
}

In [ ]:
# Choose the credentials to use.

#cos_credentials = cos_credentials_stage1_wd
cos_credentials = cos_credentials_prod_ki

#mh_credentials = mh_credentials_stage1_1z
mh_credentials = mh_credentials_stage1_2s
#mh_credentials = mh_credentials_prod_8x

In [ ]:
# Load the yelp data

import watson_streaming_pipelines as stp

bucket_name = 'pyml'
object_name = 'yelp_academic_dataset_review_first200k.json'
#object_name = 'yelp_academic_dataset_review_first1k_training.json'

# TEST TEST access
#object_name = 'test.txt'
#data_in = 'this is a test'
#resp = stp.put_to_cloud_object_storage(
#            api_key = cos_credentials['api_key'],
#            full_object_path = bucket_name + '/' + object_name, 
#            my_data = data_in,
#            auth_endpoint = cos_credentials['iam_url'],
#            service_endpoint = cos_credentials['endpoint']
#)
#print("resp = ", resp)

data = stp.get_from_cloud_object_storage( 
            api_key = cos_credentials['api_key'],
            full_object_path = bucket_name + '/' + object_name, 
            auth_endpoint = cos_credentials['iam_url'],
            service_endpoint = cos_credentials['endpoint']
) 

# Verify
print(len(data))

In [ ]:
# Convert and decode the binary data and load the yelp List with the JSON objects

from io import StringIO
import json

yelp = []
for line in StringIO(data.decode("utf-8")):
    yelp.append( json.loads(line))

# Verify
print(yelp[0]['text'])

In [ ]:
# Create the Message Hub producer
producer = stp.create_messagehub_producer( username = mh_credentials['user'], password = mh_credentials['password'], kafka_brokers_sasl = mh_credentials['kafka_brokers_sasl'])

# Enter your topic (<REPLACE>)
mh_topic = 'testTopic1'
#mh_topic = 'myTopic1'

In [ ]:
# Send selected yelp fields to the MH topic

import time
print(len(yelp))
#print(yelp[0]['stars'])
n=0
for msg in yelp: #[:1000]:
#    print(msg['stars'])
    producer.send( mh_topic, { 'stars': msg['stars'], 'text': msg['text']})
    n += 1
    if (n % 1000) == 0:
        time.sleep(1)
        print(n, "", end="")